In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow for Colab

# 画像ファイルのパス
image_path = '/content/スクリーンショット 2024-11-26 134910.png'

# 画像を読み込む
image = cv2.imread(image_path)
if image is None:
    raise ValueError("画像が読み込めませんでした。ファイルパスを確認してください。")

# 色空間をBGRからHSVに変換
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 青色の閾値を設定（盤面の端を検出）
lower_blue = np.array([100, 150, 50])
upper_blue = np.array([140, 255, 255])

# 青色領域のマスクを作成
mask_blue = cv2.inRange(hsv_image, lower_blue, upper_blue)

# マスクを基に輪郭を検出
contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 最大の輪郭（盤面の外枠）を特定
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    # 盤面を切り出し（さらに内部の領域を切り取るために余白を削除）
    margin = 10  # 余白のサイズ
    x, y, w, h = x + margin, y + margin, w - 2 * margin, h - 2 * margin
    board_image = image[y:y+h, x:x+w]
else:
    raise ValueError("盤面の外枠が検出できませんでした。青色の範囲を確認してください。")

# 切り出した盤面をリサイズ（7×6のマス目に合わせる）
resized_image = cv2.resize(board_image, (700, 600))

# 色空間をHSVに変換
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)

# 赤色と黄色の閾値を設定
lower_red1 = np.array([0, 100, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([165, 100, 50])
upper_red2 = np.array([180, 255, 255])
lower_yellow = np.array([15, 80, 80])
upper_yellow = np.array([35, 255, 255])

# 赤色と黄色のマスク
mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
mask_red = cv2.bitwise_or(mask_red1, mask_red2)
mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

# ボードのマスを縦6、横7のグリッドに分割
grid_rows, grid_cols = 6, 7
cell_height = resized_image.shape[0] // grid_rows
cell_width = resized_image.shape[1] // grid_cols

# 結果画像と出力フォーマット
result_image = resized_image.copy()
board_state = []

# 判定の処理
for row in range(grid_rows):
    row_state = []
    for col in range(grid_cols):
        # 各セルの範囲を計算
        y_start, y_end = row * cell_height, (row + 1) * cell_height
        x_start, x_end = col * cell_width, (col + 1) * cell_width

        # 各マスクのピクセル値を集計
        red_density = np.mean(mask_red[y_start:y_end, x_start:x_end])
        yellow_density = np.mean(mask_yellow[y_start:y_end, x_start:x_end])

        # 色の割合で判定
        if red_density > yellow_density:
            row_state.append('R')
            result = 'R'
            color = (0, 0, 255)  # 赤色
        elif yellow_density > red_density:
            row_state.append('Y')
            result = 'Y'
            color = (0, 255, 255)  # 黄色
        else:
            row_state.append('-')
            result = '-'
            color = (255, 255, 255)  # 白色

        # 結果を描画
        cv2.putText(result_image, result, (x_start + 10, y_start + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        # マス目の境界線を描画
        cv2.rectangle(result_image, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
    board_state.append(row_state)

# 結果をフォーマットして表示
output = "\n".join(" ".join(row) for row in board_state)
print(output)

# 画像を表示
cv2_imshow(result_image)

In [ ]:
from skimage import io, color, transform
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# 画像ファイルのパス
image_path = '/content/スクリーンショット 2024-11-26 134910.png'

# 画像を読み込む
image = io.imread(image_path)
if image is None:
    raise ValueError("画像が読み込めませんでした。ファイルパスを確認してください。")

# 画像をリサイズ（高速化のため小さくする場合もあり）
image = transform.resize(image, (600, 700), anti_aliasing=True)

# Check if the image has an alpha channel (4 channels)
if image.shape[2] == 4:
    # If so, remove the alpha channel by selecting only the first 3 channels (RGB)
    image = image[:, :, :3]

# 色空間をRGBからHSVに変換
hsv_image = color.rgb2hsv(image)

# 赤色と黄色の閾値を設定（色相に基づく範囲調整）
# Hueは0~1の範囲に正規化されているため、範囲を再調整
lower_red1 = (0.0, 0.4, 0.4)  # 最初の赤の範囲 (Hue, Saturation, Value)
upper_red1 = (0.05, 1.0, 1.0)
lower_red2 = (0.9, 0.4, 0.4)  # 第二の赤の範囲
upper_red2 = (1.0, 1.0, 1.0)
lower_yellow = (0.13, 0.4, 0.4)  # 黄色の範囲
upper_yellow = (0.2, 1.0, 1.0)

# マスクを作成
mask_red1 = ((hsv_image[:, :, 0] >= lower_red1[0]) & (hsv_image[:, :, 0] <= upper_red1[0]) &
             (hsv_image[:, :, 1] >= lower_red1[1]) & (hsv_image[:, :, 2] >= lower_red1[2]))
mask_red2 = ((hsv_image[:, :, 0] >= lower_red2[0]) & (hsv_image[:, :, 0] <= upper_red2[0]) &
             (hsv_image[:, :, 1] >= lower_red2[1]) & (hsv_image[:, :, 2] >= lower_red2[2]))
mask_red = mask_red1 | mask_red2

mask_yellow = ((hsv_image[:, :, 0] >= lower_yellow[0]) & (hsv_image[:, :, 0] <= upper_yellow[0]) &
               (hsv_image[:, :, 1] >= lower_yellow[1]) & (hsv_image[:, :, 2] >= lower_yellow[2]))

# マスクの視覚化
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(mask_red, cmap='Reds')
plt.title("Red Mask")

plt.subplot(1, 2, 2)
plt.imshow(mask_yellow, cmap='YlOrBr')
plt.title("Yellow Mask")

plt.tight_layout()
plt.show()


# マスクのデバッグ表示
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(image)

plt.subplot(1, 3, 2)
plt.title("Red Mask")
plt.imshow(mask_red, cmap='Reds')

plt.subplot(1, 3, 3)
plt.title("Yellow Mask")
plt.imshow(mask_yellow, cmap='Oranges')
plt.show()

# グリッドに分割して駒を認識
grid_rows, grid_cols = 6, 7
cell_height = image.shape[0] // grid_rows
cell_width = image.shape[1] // grid_cols

board_state = []
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image)


for row in range(grid_rows):
    row_state = []
    for col in range(grid_cols):
        y_start, y_end = row * cell_height, (row + 1) * cell_height
        x_start, x_end = col * cell_width, (col + 1) * cell_width

        red_density = np.mean(mask_red[y_start:y_end, x_start:x_end])
        yellow_density = np.mean(mask_yellow[y_start:y_end, x_start:x_end])

        if red_density > yellow_density and red_density > 0.2:  # 赤が支配的
            row_state.append('R')
            color = 'red'
        elif yellow_density > red_density and yellow_density > 0.2:  # 黄が支配的
            row_state.append('Y')
            color = 'yellow'
        else:  # 空白
            row_state.append('-')
            color = 'white'

        # 描画
        rect = Rectangle((x_start, y_start), cell_width, cell_height, linewidth=1, edgecolor='green', facecolor='none')
        ax.add_patch(rect)
        ax.text(x_start + cell_width // 3, y_start + cell_height // 2, row_state[-1], fontsize=12, color=color)
    board_state.append(row_state)

plt.axis('off')
plt.show()


# 勝敗判定

def check_winner(board, player):
    """4連続があるかを確認する"""
    rows, cols = len(board), len(board[0])
    for r in range(rows):
        for c in range(cols):
            # 横方向
            if c + 3 < cols and all(board[r][c+i] == player for i in range(4)):
                return True
            # 縦方向
            if r + 3 < rows and all(board[r+i][c] == player for i in range(4)):
                return True
            # 右下方向（斜め）
            if r + 3 < rows and c + 3 < cols and all(board[r+i][c+i] == player for i in range(4)):
                return True
            # 左下方向（斜め）
            if r + 3 < rows and c - 3 >= 0 and all(board[r+i][c-i] == player for i in range(4)):
                return True
    return False

# 勝者の判定
if check_winner(board_state, 'R'):
    print("赤の勝利！")
elif check_winner(board_state, 'Y'):
    print("黄色の勝利！")
else:
    print("勝者なし。ゲーム継続可能。")

# 結果の出力
output = "\n".join(" ".join(row) for row in board_state)
print(output)

In [ ]:
import openai
import numpy as np
from skimage import io, color, transform
import os

# OpenAI APIキーの設定
openai.api_key = "your-api-key-here"

def process_image(image_path):
    # 画像を読み込む
    image = io.imread(image_path)
    if image is None:
        raise ValueError("画像が読み込めませんでした。ファイルパスを確認してください。")

    # 画像をリサイズ（高速化のため小さくする）
    image = transform.resize(image, (600, 700), anti_aliasing=True)

    # アルファチャンネルを削除
    if image.shape[2] == 4:
        image = image[:, :, :3]

    # 色空間をHSVに変換
    hsv_image = color.rgb2hsv(image)

    # 赤色と黄色の閾値を設定
    lower_red1, upper_red1 = (0.0, 0.4, 0.4), (0.05, 1.0, 1.0)
    lower_red2, upper_red2 = (0.9, 0.4, 0.4), (1.0, 1.0, 1.0)
    lower_yellow, upper_yellow = (0.13, 0.4, 0.4), (0.2, 1.0, 1.0)

    # マスクを作成
    mask_red1 = ((hsv_image[:, :, 0] >= lower_red1[0]) & (hsv_image[:, :, 0] <= upper_red1[0]) &
                 (hsv_image[:, :, 1] >= lower_red1[1]) & (hsv_image[:, :, 2] >= lower_red1[2]))
    mask_red2 = ((hsv_image[:, :, 0] >= lower_red2[0]) & (hsv_image[:, :, 0] <= upper_red2[0]) &
                 (hsv_image[:, :, 1] >= lower_red2[1]) & (hsv_image[:, :, 2] >= lower_red2[2]))
    mask_red = mask_red1 | mask_red2

    mask_yellow = ((hsv_image[:, :, 0] >= lower_yellow[0]) & (hsv_image[:, :, 0] <= upper_yellow[0]) &
                   (hsv_image[:, :, 1] >= lower_yellow[1]) & (hsv_image[:, :, 2] >= lower_yellow[2]))

    # グリッドに分割して駒を認識
    grid_rows, grid_cols = 6, 7
    cell_height = image.shape[0] // grid_rows
    cell_width = image.shape[1] // grid_cols

    board_state = []
    for row in range(grid_rows):
        row_state = []
        for col in range(grid_cols):
            y_start, y_end = row * cell_height, (row + 1) * cell_height
            x_start, x_end = col * cell_width, (col + 1) * cell_width

            red_density = np.mean(mask_red[y_start:y_end, x_start:x_end])
            yellow_density = np.mean(mask_yellow[y_start:y_end, x_start:x_end])

            if red_density > yellow_density and red_density > 0.2:
                row_state.append('R')
            elif yellow_density > red_density and yellow_density > 0.2:
                row_state.append('Y')
            else:
                row_state.append('-')
        board_state.append(row_state)
    return board_state

def send_to_chatgpt(board_state):
    # 盤面データを文字列形式に変換
    board_str = "\n".join(" ".join(row) for row in board_state)

    # ChatGPTへのプロンプト
    prompt = f"""
以下の四目並べの盤面を解析し、現在の状態を説明してください。盤面形式は以下の通りです：
{board_str}
- は空白を示します。R は赤、Y は黄色の駒を示します。
"""

    # ChatGPT APIへのリクエスト
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].text.strip()

# メイン処理
image_path = "your-image-path.png"  # 画像のパスを指定
if os.path.exists(image_path):
    board_state = process_image(image_path)
    print("解析結果:")
    for row in board_state:
        print(" ".join(row))

    # ChatGPTに送信
    analysis = send_to_chatgpt(board_state)
    print("\nChatGPTの解析:")
    print(analysis)
else:
    print("指定された画像ファイルが存在しません。")


In [ ]:
def majority_vote(boards):
    """
    各プログラムの結果から多数決で最終盤面を作成する。
    :param boards: 各プログラムが出力した盤面のリスト (3つの6x7のリスト)
    :return: 多数決で判定された盤面
    """
    grid_rows, grid_cols = len(boards[0]), len(boards[0][0])
    final_board = []

    for row in range(grid_rows):
        final_row = []
        for col in range(grid_cols):
            # 各プログラムのセルの値を収集
            votes = [boards[i][row][col] for i in range(len(boards))]
            # 多数決で最頻値を決定
            most_common = max(set(votes), key=votes.count)
            final_row.append(most_common)
        final_board.append(final_row)

    return final_board


def check_winner(board, player):
    """
    4連続があるかを確認する。
    :param board: 最終盤面
    :param player: 'R' または 'Y'
    :return: True if the player wins, otherwise False
    """
    rows, cols = len(board), len(board[0])
    for r in range(rows):
        for c in range(cols):
            # 横方向
            if c + 3 < cols and all(board[r][c + i] == player for i in range(4)):
                return True
            # 縦方向
            if r + 3 < rows and all(board[r + i][c] == player for i in range(4)):
                return True
            # 右下方向（斜め）
            if r + 3 < rows and c + 3 < cols and all(board[r + i][c + i] == player for i in range(4)):
                return True
            # 左下方向（斜め）
            if r + 3 < rows and c - 3 >= 0 and all(board[r + i][c - i] == player for i in range(4)):
                return True
    return False

# 各プログラムの出力例 (6x7の盤面状態)
board1 = [
    ['-', '-', '-', '-', '-', '-', '-'],
    ['-', '-', '-', '-', '-', '-', '-'],
    ['-', '-', 'R', '-', '-', '-', '-'],
    ['-', '-', 'Y', 'R', '-', '-', '-'],
    ['-', '-', 'Y', 'Y', 'R', '-', '-'],
    ['R', 'R', 'Y', 'R', 'Y', 'R', '-']
]

board2 = [
    ['-', '-', '-', '-', '-', '-', '-'],
    ['-', '-', '-', '-', '-', '-', '-'],
    ['-', '-', 'R', '-', '-', '-', '-'],
    ['-', '-', 'Y', 'R', '-', '-', '-'],
    ['-', '-', 'Y', 'Y', 'R', '-', '-'],
    ['R', 'R', 'Y', 'R', 'Y', 'R', '-']
]

board3 = [
    ['-', '-', '-', '-', '-', '-', '-'],
    ['-', '-', '-', '-', '-', '-', '-'],
    ['-', '-', 'R', '-', '-', '-', '-'],
    ['-', '-', 'Y', 'R', '-', '-', '-'],
    ['-', '-', 'Y', 'Y', 'R', '-', '-'],
    ['R', 'R', 'Y', 'R', 'Y', 'R', '-']
]

# 各プログラムの盤面結果をリストにまとめる
boards = [board1, board2, board3]

# 多数決で最終盤面を作成
final_board = majority_vote(boards)

# 最終盤面の出力
print("最終盤面:")
for row in final_board:
    print(" ".join(row))

# 勝者の判定
if check_winner(final_board, 'R'):
    print("赤の勝利！")
elif check_winner(final_board, 'Y'):
    print("黄色の勝利！")
else:
    print("勝者なし。ゲーム継続可能。")
